<a href="https://colab.research.google.com/github/Heloput/machinelearning/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2024-12-25 13:17:12--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2024-12-25 13:17:13--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  9.53MB/s    in 0.3s    

2024-12-25 13:17:15 (9.53 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/

In [2]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-19.png', 'happy1-03.png', 'happy2-11.png', 'happy1-16.png', 'happy2-07.png', 'happy1-19.png', 'happy1-02.png', 'happy2-13.png', 'happy2-16.png', 'happy1-00.png']
['sad2-10.png', 'sad2-03.png', 'sad2-09.png', 'sad1-05.png', 'sad1-06.png', 'sad2-13.png', 'sad1-04.png', 'sad1-07.png', 'sad1-18.png', 'sad1-11.png']


In [3]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-19.png', 'happy1-03.png', 'happy2-11.png', 'happy1-16.png', 'happy2-07.png', 'happy1-19.png', 'happy1-02.png', 'happy2-13.png', 'happy2-16.png', 'happy1-00.png']
['sad2-10.png', 'sad2-03.png', 'sad2-09.png', 'sad1-05.png', 'sad1-06.png', 'sad2-13.png', 'sad1-04.png', 'sad1-07.png', 'sad1-18.png', 'sad1-11.png']


In [ ]:
DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("accuracy") >= DESIRED_ACCURACY:
            print(f"\nДостигнута точность {DESIRED_ACCURACY * 100:.1f}%, поэтому обучение закончено!")
            self.model.stop_training = True

callbacks = myCallback()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')

history = model.fit(
      train_generator,
      steps_per_epoch=(int(len(train_generator)/16.0)),
      epochs=100,  # Увеличьте количество эпох, если данных мало
      verbose=1,
      callbacks=[callbacks]
)

print(history.epoch, history.history['accuracy'][-1])


Found 80 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 294ms/step - accuracy: 0.3825 - loss: 0.8254
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step - accuracy: 0.5667 - loss: 0.7032
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 288ms/step - accuracy: 0.7394 - loss: 0.6810
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 478ms/step - accuracy: 0.6141 - loss: 0.6724
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 293ms/step - accuracy: 0.6884 - loss: 0.6427
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 294ms/step - accuracy: 0.7071 - loss: 0.5882
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 283ms/step - accuracy: 0.7297 - loss: 0.5310
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - accuracy: 0.6774 - loss: 0.6047
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 490ms/step - accuracy: 0.8083 - loss: 0.4028
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 289ms/step - accuracy: 0.5936 - loss: 0.6952
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 285ms/step - accuracy: 0.8557 - loss: 0.3700
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 288ms/step - accuracy: 0.8257 - lo